## CURSO DE CAPACITACIÓN: ANÁLISIS DE IMÁGENES DIGITALES PARA EL FENOTIPADO VEGETAL

### FLUJO DE TRABAJO PARA CONTAR Y MEDIR SEMILLAS

## Sección 1. Importación de librerías e imagen

In [ ]:
#Importanción de librerías
import os
#import argparse
#import matplotlib
import numpy as np
from plantcv.parallel import WorkflowInputs
from plantcv import plantcv as pcv
import pandas as pd




In [ ]:

# Opciones de archivos de entrada 
args = WorkflowInputs(
    images=["./imagenes/espinaca.jpeg"],
    names="image",
    result="./res.csv",
    outdir=".",
    writeimg=False,
    debug="plot"
    )

# Set debug to the global parameter 
pcv.params.debug = args.debug

# Change display settings
pcv.params.dpi = 170
#pcv.params.text_size = 10
#pcv.params.text_thickness = 20



# Update params text size
pcv.params.text_size=1
pcv.params.text_thickness=1

img, path, filename = pcv.readimage(filename=args.image)


In [ ]:
# Spot the card and standarize image color
card_mask = pcv.transform.detect_color_card(rgb_img=img, adaptive_method=1, block_size=31, radius=20)
headers, card_matrix = pcv.transform.get_color_matrix(rgb_img=img, mask=card_mask)




# Define the standard color card matrix, we know what the colors of those chips should be in an "ideal" image, 
# so we will correct to those values as the TARGET c
# Look at where your white chip is in the image to determine which position your card is in (pos)

#pos     = reference value indicating orientation of the color card. The reference
       #         is based on the position of the white chip:
        #        pos = 0: bottom-left corner
        #        pos = 1: bottom-right corner
        #        pos = 2: top-right corner
        #        pos = 3: top-left corner

std_color_matrix = pcv.transform.std_color_matrix(pos=3)

img_cc = pcv.transform.affine_color_correction(rgb_img=img, 
                                               source_matrix=card_matrix,
                                               target_matrix=std_color_matrix)

pcv.plot_image(img_cc)

## Sección 2. Segmentación e identificación de objetos

### Visualización de los diversos espacios de color

In [ ]:
# Inputs:
#   rbg_img      = original image
#   original_img = whether to include the original RGB images in the display: True (default) or False
colorspace_img = pcv.visualize.colorspaces(rgb_img=img_cc)

### Conversión de la imagen a escala de grises

In [ ]:
# Inputs:
#   rbg_img - original image
#   channel - desired colorspace ('h', 's', or 'v')
a_img = pcv.rgb2gray_hsv(rgb_img=img_cc, channel='s')


In [ ]:
#Binarize, remove salt and pepper, and fill areas

th_otsu=pcv.threshold.otsu(gray_img=a_img, object_type='light')
a_erode=pcv.erode(gray_img=th_otsu, ksize=2, i=1)
a_dilation=pcv.dilate(gray_img=a_erode, ksize=2, i=1)
mask_fill = pcv.fill(bin_img=a_dilation, size=1)
mask_fill = pcv.fill_holes(bin_img=mask_fill)



In [ ]:
#Image dimensions
print(img_cc.shape)

In [ ]:
#ROI rectangular

roi1 = pcv.roi.rectangle(img=img_cc, x=400, y=0, h=700, w=600)



In [ ]:
kept_mask  = pcv.roi.filter(mask=mask_fill, roi=roi1, roi_type='partial')


In [ ]:
# Watershed segmentation
'''
Parameters:
rgb_img - RGB image data
mask - Binary image, single channel, object in white and background black
distance - Minimum distance of local maximum, lower values are more sensitive, and segments more objects (default: 10)
label - Optional label parameter, modifies the variable name of observations recorded. (default = pcv.params.sample_label)
'''
watershed_labels=pcv.watershed_segmentation(rgb_img=img_cc, mask=kept_mask, distance=5)

#Get the number of objects
n_obj_wshed=np.unique(watershed_labels)[-1]-1

# Note: Some merged seeds have been separated

In [ ]:
#Another option


#Label the objects in your region of interest and number them
#Each object should be a different color. If your objects are the same color and/or are touching, go back to your mask so that they are separate or it will treat them as a single object

#    mask            = the clean mask you made above after making your ROI

labeled_objects, n_obj = pcv.create_labels(mask=kept_mask)

print(n_obj)

img_semillas=pcv.apply_mask(img=img, mask=kept_mask,mask_color='white')

## Sección 3. Análisis morfológico de semillas

Para esto se necesita una máscara binaria completa


### Identificación simple de objetos

La máscara binaria se usa para encontrar objetos o contornos de cada una de las semillas. A diferencia de la función find_objects, esta aplicación usa la función findContours del programa OpenCV con la entrada cv2.RETR_EXTERNAL que ignora contornos en capas. La salida de este paso se puede usar para contar semillas, pero no se puede usar como entrada para análisis de color o forma.

In [ ]:
############### Analysis ################ 
  
# Find shape properties, data gets stored to an Outputs class automatically

# Inputs:
#   img - RGB or grayscale image data 
#   labeled_mask - the mask of each individual object, set by the create_labels function. 
#   n_labels - the number of objects, set by the create_labels function. 

analysis_image = pcv.analyze.size(img=img_cc, labeled_mask=watershed_labels, n_labels=n_obj_wshed)

In [ ]:
#Manually Obtain morphological values 
chip_length=12 #mm

avg_chip_size = pcv.outputs.metadata['median_color_chip_size']['value'][0] #px
chip_width_px=pcv.outputs.metadata['median_color_chip_width']['value'][0] # px
chip_height_px=pcv.outputs.metadata['median_color_chip_height']['value'][0] # px
chip_length_avg=(chip_width_px+chip_height_px)/2 # cm
ratio_length=chip_length/chip_length_avg 

ratio_area=(chip_length**2)/chip_length_avg**2
#ratio_area=(chip_length**2)/avg_chip_size

print(f'Área del chip en píxels: {avg_chip_size:.1f}')
print(f'Altura del chip en píxeles: {chip_height_px:.2f}')
print(f'Ancho del chip en píxeles: {chip_width_px:.2f}')
print(f'Dimensión promedio del chip en píxeles: {chip_length_avg:.2f}')
print(f'Relación mm/píxel de la longitud promedio del chip: {ratio_length:.4f}')
print(f'Relación mm2/píxeles de la superficie del chip: {ratio_area:.6f}')

In [ ]:
!pip install openpyxl

In [ ]:
# Define the column titles
column_titles = ['archivo','n_obj','largo_mm', 'ancho_mm', 'area_mm2','perimetro_mm','solidez']

# Create an empty DataFrame with the specified columns
data = pd.DataFrame(columns=column_titles)


for i in range(n_obj_wshed):
    height=pcv.outputs.observations[f'default_{i+1}']['height']['value']*ratio_length
    width=pcv.outputs.observations[f'default_{i+1}']['width']['value']*ratio_length
    area = pcv.outputs.observations[f'default_{i+1}']['area']['value']*ratio_area
    perimeter = pcv.outputs.observations[f'default_{i+1}']['perimeter']['value']*ratio_length
    solidity = pcv.outputs.observations[f'default_{i+1}']['solidity']['value']
    data.loc[i]= [filename,i,height,width,area,perimeter,solidity]
    
archivo='./res_semillas.xlsx'

# Create the directory if it doesn't exist
directory = os.path.dirname(archivo)
if directory and not os.path.exists(directory):
    os.makedirs(directory, exist_ok=True)


# Convert columns to numeric, forcing errors to NaN, then round
for col in ['largo_mm', 'ancho_mm', 'area_mm2','perimetro_mm','solidez']:
    data[col] = pd.to_numeric(data[col], errors='coerce').map(lambda x: round(x, 2) if pd.notnull(x) else x)


# Check if the file exists
if os.path.exists(archivo):
    # If the file exists, load the workbook and append to the 'Results' sheet if it exists
    with pd.ExcelWriter(archivo, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        # Check if 'Results' sheet already exists
        if 'Results' in writer.book.sheetnames:
            # Get the maximum row in the existing sheet to append the new data below it
            startrow = writer.sheets['Results'].max_row
        else:
            # If the 'Results' sheet does not exist, start from the first row
            startrow = 0
        # Write the DataFrame to the existing file, appending data if the sheet exists <----- Key procedure!
        data.to_excel(writer, sheet_name='Results', header=startrow == 0, startrow=startrow, index=True)
else:
    # If the file does not exist, create it and write the data
    with pd.ExcelWriter(archivo, engine='openpyxl') as writer:
        data.to_excel(writer, sheet_name='Results', index=True)


data
